In [0]:
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [0]:
# Hyper parameters
batch_size = 128
nb_epoch = 10

# Parameters for MNIST dataset
nb_classes = 10

# Parameters for MLP
prob_drop_input = 0.2               # drop probability for dropout @ input layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

In [0]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_Train = np_utils.to_categorical(y_train, nb_classes)
Y_Test = np_utils.to_categorical(y_test, nb_classes)

In [15]:
# Multilayer Perceptron model
model = Sequential()
model.add(Dense(units=625, input_dim=784, kernel_initializer='normal', activation='sigmoid', name='dense1'))
model.add(Dropout(prob_drop_input, name='dropout1'))
model.add(Dense(units=625, input_dim=625, kernel_initializer='normal', activation='sigmoid', name='dense2'))
model.add(Dropout(prob_drop_hidden, name='dropout2'))
model.add(Dense(units=10, input_dim=625, kernel_initializer='normal', activation='softmax', name='dense3'))
model.compile(optimizer=RMSprop(lr=0.001, rho=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 625)               490625    
_________________________________________________________________
dropout1 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense2 (Dense)               (None, 625)               391250    
_________________________________________________________________
dropout2 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense3 (Dense)               (None, 10)                6260      
Total params: 888,135
Trainable params: 888,135
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Train
save_model(model, './logs/model_mlp')
checkpoint = ModelCheckpoint(filepath='./logs/weights.epoch.{epoch:02d}-val_loss.{val_loss:.2f}.hdf5', verbose=0)
history = model.fit(X_train, Y_Train, epochs=nb_epoch, batch_size=batch_size, verbose=1,
                    callbacks=[checkpoint], validation_data=(X_test, Y_Test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 69us/step - loss: 0.5704 - acc: 0.8209 - val_loss: 0.2568 - val_acc: 0.9214
Epoch 2/10
60000/60000 [==============================] - 4s 69us/step - loss: 0.2499 - acc: 0.9250 - val_loss: 0.1758 - val_acc: 0.9454
Epoch 3/10
60000/60000 [==============================] - 4s 65us/step - loss: 0.1838 - acc: 0.9444 - val_loss: 0.1389 - val_acc: 0.9571
Epoch 4/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.1444 - acc: 0.9564 - val_loss: 0.1144 - val_acc: 0.9649
Epoch 5/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.1192 - acc: 0.9633 - val_loss: 0.0992 - val_acc: 0.9690
Epoch 6/10
60000/60000 [==============================] - 4s 70us/step - loss: 0.1027 - acc: 0.9691 - val_loss: 0.0930 - val_acc: 0.9723
Epoch 7/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0925 - acc: 0.9714 - val_loss: 0.0862 - val_acc

In [17]:
# Evaluate
evaluation = model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

10000/10000 [==============================] - 1s 81us/step

Summary: Loss over the test dataset: 0.08, Accuracy: 0.98


In [18]:
# Restore trained model
loaded_model = load_model('./logs/model_mlp')
loaded_model.load_weights('./logs/weights.epoch.09-val_loss.0.08.hdf5')
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 625)               490625    
_________________________________________________________________
dropout1 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense2 (Dense)               (None, 625)               391250    
_________________________________________________________________
dropout2 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense3 (Dense)               (None, 10)                6260      
Total params: 888,135
Trainable params: 888,135
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Evaluate with loaded model
evaluation = loaded_model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

10000/10000 [==============================] - 1s 87us/step

Summary: Loss over the test dataset: 0.08, Accuracy: 0.98
